# Icon Semantic Classification

This notebook is heavily based on TensorFlow guides to [Image classification](https://www.tensorflow.org/tutorials/images/classification) and [Transfer learning and fine-tuning](https://www.tensorflow.org/tutorials/images/transfer_learning) which are published under Apache License 2.0.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import seaborn as sns
from sklearn.metrics import classification_report
import json

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (96, 96)
IMG_SHAPE = IMG_SIZE + (3,)
INITIAL_EPOCHS = 20
FINE_TUNE_EPOCHS = 20

## Load Dataset

load the dataset ($D_\text{mobile}$ or $D_\text{all}$) from disk

In [ ]:
dataset_path = Path("<path_to_dataset>")

ds_train = tf.keras.utils.image_dataset_from_directory(
    dataset_path / "training",
    shuffle=True,
    seed=42,
    color_mode="rgb",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
)

ds_validation = tf.keras.utils.image_dataset_from_directory(
    dataset_path / "validation",
    shuffle=True,
    seed=42,
    color_mode="rgb",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
)

class_names = ds_train.class_names
num_classes = len(class_names)

# performance tuning
AUTOTUNE = tf.data.AUTOTUNE
ds_train = ds_train.prefetch(buffer_size=AUTOTUNE)
ds_validation = ds_validation.prefetch(buffer_size=AUTOTUNE)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in ds_train.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i].numpy().argmax()])
        plt.axis("off")
plt.show()

## Data Preprocessing

rescale images from $[0, 255]$ to $[-1, 1]$

In [ ]:
preprocessing = tf.keras.layers.Rescaling(1.0 / 127.5, offset=-1)

## Data Augmentation

apply random contrast, translation and zoom to the image

In [ ]:
augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomContrast(0.1),
        tf.keras.layers.RandomTranslation(0.1, 0.1),
        tf.keras.layers.RandomZoom((-0.05, 0)),
    ]
)

In [ ]:
for image, _ in ds_train.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = augmentation(tf.expand_dims(first_image, 0), training=True)
        plt.imshow(augmented_image[0] / 255)
        plt.axis("off")

## Model

In [ ]:
mobilenetv2 = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SHAPE), include_top=False, weights="imagenet"
)

# freeze weights
mobilenetv2.trainable = False

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = augmentation(inputs)
x = preprocessing(x)
x = mobilenetv2(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(num_classes)(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
# alternativly the baseline model can be used here.
# if the baseline model is used the fine tuning step can be skipped.
# baseline_cnn = tf.keras.models.Sequential(
#     [
#         tf.keras.Input(shape=IMG_SHAPE),
#         data_augmentation,
#         tf.keras.layers.Rescaling(1.0 / 127.5, offset=-1),
#         tf.keras.layers.Conv2D(16, 3, padding="same", activation="relu"),
#         tf.keras.layers.MaxPooling2D(),
#         tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
#         tf.keras.layers.MaxPooling2D(),
#         tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
#         tf.keras.layers.MaxPooling2D(),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Flatten(),
#         tf.keras.layers.Dense(128, activation="relu"),
#         tf.keras.layers.Dense(num_classes),
#     ]
# )

## Training

### Pre Fine Tuning

In [ ]:
base_learning_rate = 0.0001
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
model.summary()

In [ ]:
history = model.fit(ds_train, epochs=INITIAL_EPOCHS, validation_data=ds_validation)

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label="Training Accuracy")
plt.plot(val_acc, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.ylabel("Accuracy")
plt.ylim([min(plt.ylim()), 1])
plt.title("Training and Validation Accuracy")

plt.subplot(2, 1, 2)
plt.plot(loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.ylabel("Cross Entropy")
plt.ylim([0, 1.0])
plt.title("Training and Validation Loss")
plt.xlabel("epoch")
plt.show()

### Fine Tuning

In [ ]:
# unfreeze weights
mobilenetv2.trainable = True

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate / 10),
    metrics=["accuracy"],
)

In [ ]:
model.summary()

In [ ]:
total_epochs = INITIAL_EPOCHS + FINE_TUNE_EPOCHS

history_fine = model.fit(
    ds_train,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=ds_validation,
)

In [ ]:
acc += history_fine.history["accuracy"]
val_acc += history_fine.history["val_accuracy"]

loss += history_fine.history["loss"]
val_loss += history_fine.history["val_loss"]

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label="Training Accuracy")
plt.plot(val_acc, label="Validation Accuracy")
plt.ylim([0.8, 1])
plt.plot(
    [INITIAL_EPOCHS - 1, INITIAL_EPOCHS - 1], plt.ylim(), label="Start Fine Tuning"
)
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(2, 1, 2)
plt.plot(loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.ylim([0, 1.0])
plt.plot(
    [INITIAL_EPOCHS - 1, INITIAL_EPOCHS - 1], plt.ylim(), label="Start Fine Tuning"
)
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.xlabel("epoch")
plt.show()

In [ ]:
model.save("./model.h5")

## Evaluation

evaluate the models accuracy, precision, recall, f1-score and confusion matrix (e.g. on $D_\text{web}$).

In [ ]:
# ds_test = tf.keras.utils.image_dataset_from_directory(
#     Path("<path_to_test_set>"),
#     shuffle=True,
#     seed=42,
#     color_mode="rgb",
#     label_mode="categorical",
#     batch_size=BATCH_SIZE,
#     image_size=IMG_SIZE,
# )
ds_test = ds_validation

In [ ]:
test_loss, test_accuracy = model.evaluate(ds_test)

print("Loss: {:.2f}".format(test_loss))
print("Accuracy: {:.2f}".format(test_accuracy))

In [ ]:
y_pred = np.array([], dtype="float32")
y_true = np.array([], dtype="float32")

for images, labels in ds_test:
    pred = tf.nn.softmax(model.predict(images)).numpy().argmax(axis=1)
    actual = labels.numpy().argmax(axis=1)
    y_pred = np.concatenate([y_pred, pred])
    y_true = np.concatenate([y_true, actual])

In [ ]:
report = classification_report(
    y_true, y_pred, target_names=class_names, output_dict=True
)
with open("./classification_report.json", "w") as f:
    f.write(json.dumps(report, indent=2))

In [ ]:
confusion_matrix = tf.math.confusion_matrix(y_true, y_pred)
sns.heatmap(
    confusion_matrix,
    xticklabels=class_names,
    yticklabels=class_names,
    annot=True,
    fmt="g",
)
plt.xlabel("Prediction")
plt.ylabel("Actual")
plt.savefig("confusion_matrix.png", dpi=200)
plt.show()

## Export model for TensorFlow.js

To load the model in TensorFlow.js we have to remove the preprocessing and data augmentation layers as they are not supported in TensorFlow.js.

In [ ]:
tfjs_model = tf.keras.models.Sequential(
    [tf.keras.Input(shape=IMG_SHAPE), *model.layers[3:]]
)

In [ ]:
tfjs_model.summary()

In [ ]:
# test if output is still the same
test_image = tf.random.uniform((1, *IMG_SHAPE))
rescaled_test_image = preprocessing(test_image)
print(tf.equal(model(test_image),  tfjs_model(rescaled_test_image)))

In [ ]:
tfjs_model.save("./tfjs_model.h5")